# Item-based Models

In this file we are going to proceed with the building in the models. Machine learning models to build recommender systems. The models that are going to be built are Collaborative filtering using Item-based rating prediction (ItemKNN) and Item-based classification (ItemKNN).

Collaborative filtering is a technique used in recommendation systems to predict or classify items based on the preferences or behavior of similar users or items. Item-based Collaborative Filtering (CF) focuses on the similarity between items rather than users. There are two main approaches within item-based CF: rating prediction and classification.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

In [2]:
# We import the necessary libraries
import pandas as pd
import numpy as np
import os

In [3]:
# We print the directory where the file is located
print(os.getcwd())

c:\Users\Jaume\Documents\MDDB\SDM\SDfM---Jaume-and-Stijn 2\SDfM---Jaume-and-Stijn


In [4]:
# We set the directory to the cleaned folder
os.listdir(os.path.join('.', 'cleaned'))

['final_sample30_parquet',
 'final_sample50_parquet',
 'final_sample5_parquet',
 'final_strat_sample_parquet',
 'movielens_parquet',
 'movielens_parquet_small',
 'netflix_parquet',
 'sample_tenth_netflix',
 'strat_sample_movielens',
 'strat_sample_netflix']

In [5]:
# We read the final_sample file and store it in a dataframe
df = pd.read_parquet('cleaned/final_strat_sample_parquet')

In [6]:
# We print shape of the dataframe
df.shape

(50381, 25)

In [7]:
# We print the first 5 rows of the dataframe
df.head()

,movieId,title,year,userId,rating,date,Drama,Thriller,Crime,Mystery,...,Musical,Romance,Sci-Fi,Children,Action,Horror,War,IMAX,Western,(no genres listed)
0,143367,Silence,2016,267711,3.5,2022-05-01,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,143367,Silence,2016,9138,4.5,2021-08-18,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,143367,Silence,2016,9138,4.5,2021-08-18,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,143367,Silence,2016,306765,3.5,2017-11-06,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,143367,Silence,2016,318227,2.5,2019-05-03,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'title', 'year', 'userId', 'rating', 'date', 'Drama',
       'Thriller', 'Crime', 'Mystery', 'Documentary', 'Adventure', 'Animation',
       'Comedy', 'Fantasy', 'Musical', 'Romance', 'Sci-Fi', 'Children',
       'Action', 'Horror', 'War', 'IMAX', 'Western', '(no genres listed)'],
      dtype='object')

In [9]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())


37841
500


In [10]:
# Check if there are userIDs that haven't rated movies at all
print(df['userId'].value_counts().sort_values(ascending=False))

# Check if there are movieIDs that haven't been rated at all
print(df['movieId'].value_counts().sort_values(ascending=False))

userId
215490    378
137062     43
9138       33
189614     31
222223     26
         ... 
141435      1
215467      1
114372      1
245678      1
158343      1
Name: count, Length: 37841, dtype: int64
movieId
1221      5328
733       4590
48516     4190
208       3747
89745     3326
          ... 
92760        1
119165       1
184849       1
229975       1
103570       1
Name: count, Length: 500, dtype: int64


In [11]:
# Check if there are NaN values in the rating column
df['rating'].isna().sum()

0

In [12]:
# Print the number of unique users
print("Number of unique users:", df['userId'].nunique())

# Print the number of unique movies
print("Number of unique movies:", df['movieId'].nunique())


Number of unique users: 37841
Number of unique movies: 500


In [13]:
# print the data types of the DF
df.dtypes

movieId                 int64
title                  object
year                   object
userId                  int64
rating                float64
date                   object
Drama                   int64
Thriller                int64
Crime                   int64
Mystery                 int64
Documentary             int64
Adventure               int64
Animation               int64
Comedy                  int64
Fantasy                 int64
Musical                 int64
Romance                 int64
Sci-Fi                  int64
Children                int64
Action                  int64
Horror                  int64
War                     int64
IMAX                    int64
Western                 int64
(no genres listed)      int64
dtype: object

In [15]:
# Group the DataFrame by 'userId' and count the number of ratings per user
user_rating_counts = df.groupby('userId').size()

# Filter out users who have rated less than 3 movies
active_users = user_rating_counts[user_rating_counts >= 3].index.tolist()

# Select rows from the original DataFrame for active users
df = df[df['userId'].isin(active_users)]

# Display the filtered DataFrame
df.head()

,movieId,title,year,userId,rating,date,Drama,Thriller,Crime,Mystery,...,Musical,Romance,Sci-Fi,Children,Action,Horror,War,IMAX,Western,(no genres listed)
0,143367,Silence,2016,267711,3.5,2022-05-01,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,143367,Silence,2016,9138,4.5,2021-08-18,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,143367,Silence,2016,9138,4.5,2021-08-18,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,143367,Silence,2016,306765,3.5,2017-11-06,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,143367,Silence,2016,58551,3.0,2017-01-27,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# Item-based Rating Prediction (ItemKNN)

### Step 1: User-Item matrix construction
The first thing we have to do is build the user-item matrix:

- Choose a similarity metric to calculate the similarity between items. Common metrics include cosine similarity, Pearson correlation coefficient, and Jaccard similarity.
- Calculate the similarity between each pair of items based on the ratings provided by users. This will result in an item-item similarity matrix.

##### Libraries Used
- **pandas**: A powerful data manipulation library in Python.
- **numpy**: A library for numerical computing in Python.

In [16]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame with the specified format
# Convert userId column to integer type if needed
if df['userId'].dtype != int:
    df['userId'] = df['userId'].astype(int)

# Get all unique user IDs and movie IDs
all_user_ids = np.unique(df['userId'])
all_movie_ids = np.unique(df['movieId'])

# Create a DataFrame with all combinations of user IDs and movie IDs
all_user_movie_pairs = np.array(np.meshgrid(all_user_ids, all_movie_ids)).T.reshape(-1, 2)
df_pairs = pd.DataFrame(all_user_movie_pairs, columns=['userId', 'movieId'])

# Merge the original DataFrame with all_user_movie_pairs to fill missing ratings with 0
df_filled = pd.merge(df_pairs, df, on=['userId', 'movieId'], how='left').fillna(0)

# Create user-item matrix using pivot_table
item_user_matrix = pd.pivot_table(df_filled, values='rating', index='userId', columns='movieId', fill_value=0)

# Convert user-item matrix to NumPy array for faster computation
item_user_array = item_user_matrix.to_numpy()


In [17]:
item_user_matrix.head()

movieId,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
userId,,,,,,,,,,,,,,,,,,,,,
149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
677,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
item_user_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Train-test split

The train-val-test split is a technique used in machine learning to evaluate the performance of a model. It involves dividing the dataset into three subsets: the training set, the validation set, and the test set.

The training set is used to train the model and optimize its parameters.
The validation set is used to fine-tune the model and select the best hyperparameters.
The test set is used to evaluate the final performance of the model on unseen data.
By using a train-val-test split, we can assess the model's performance on unseen data and ensure that it generalizes well to new examples. It helps prevent overfitting and provides a more reliable estimate of the model's performance in real-world scenarios.

In [19]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
train_val, test = train_test_split(item_user_array, test_size=0.2, random_state=42)

# Split the training set into training and validation sets
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

# Print the shapes of the datasets
print("Training set shape:", train.shape)
print("Validation set shape:", val.shape)
print("Test set shape:", test.shape)

# We are also going to do the split for the matrix df
# Split the user-item matrix into training and test sets
train_val_matrix, test_matrix = train_test_split(item_user_matrix, test_size=0.2, random_state=42)

# Split the training set matrix into training and validation sets
train_matrix, val_matrix = train_test_split(train_val_matrix, test_size=0.2, random_state=42)

# We print a ' ' to give some space inbetween lines
print(' ')

# Print the shapes of the matrix datasets
print("Training set matrix shape:", train_matrix.shape)
print("Validation set matrix shape:", val_matrix.shape)
print("Test set matrix shape:", test_matrix.shape)


Training set shape: (1516, 435)
Validation set shape: (379, 435)
Test set shape: (474, 435)
 
Training set matrix shape: (1516, 435)
Validation set matrix shape: (379, 435)
Test set matrix shape: (474, 435)


In [20]:
train_matrix

movieId,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
userId,,,,,,,,,,,,,,,,,,,,,
214831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
128377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232128,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
# Calculate cosine similarity between items using NumPy functions
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

# Calculate item-item similarity matrix for train data
#item_similarity_matrix_train = np.zeros((train.shape[1], train.shape[1]))

#for i in range(train.shape[1]):
 #   for j in range(i, train.shape[1]):
  #      item_similarity_matrix_train[i, j] = cosine_similarity(train[:, i], train[:, j])
   #     item_similarity_matrix_train[j, i] = item_similarity_matrix_train[i, j]

# Adjusting the mapping to start indexing from 0
user_id_to_index = {user_id: i for i, user_id in enumerate(item_user_matrix.index)}
index_to_user_id = {i: user_id for i, user_id in enumerate(item_user_matrix.index)}

# Create a mapping from movie IDs to indices
movie_id_to_index = {movie_id: i for i, movie_id in enumerate(item_user_matrix.columns)}
index_to_movie_id = {i: movie_id for i, movie_id in enumerate(item_user_matrix.columns)}




#### Collaborative Filtering with Cosine Similarity

This code snippet demonstrates how to perform item-based collaborative filtering using cosine similarity. Collaborative filtering is a technique commonly used in recommendation systems to predict a user's preferences for items based on the preferences of similar users/items.


In [23]:
# Step 2: Calculate the item-item similarity matrix for the train set
def calculate_item_similarity_matrix(data):
    item_similarity_matrix = np.zeros((data.shape[1], data.shape[1]))

    for i in range(data.shape[1]):
        for j in range(i, data.shape[1]):
            item_similarity_matrix[i, j] = cosine_similarity(data[:, i], data[:, j])
            item_similarity_matrix[j, i] = item_similarity_matrix[i, j]

    return item_similarity_matrix

item_similarity_matrix_train = calculate_item_similarity_matrix(train)

C:\Users\Jaume\AppData\Local\Temp\ipykernel_9384\2934347206.py:6: RuntimeWarning: invalid value encountered in scalar divide
  similarity = dot_product / (norm_a * norm_b)


In [24]:
item_similarity_matrix_train

array([[1.        , 0.        , 0.04116237, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04116237, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [25]:
movie_id_to_index

{75: 0,
 77: 1,
 135: 2,
 208: 3,
 290: 4,
 413: 5,
 461: 6,
 733: 7,
 792: 8,
 810: 9,
 857: 10,
 901: 11,
 955: 12,
 1184: 13,
 1221: 14,
 1352: 15,
 1365: 16,
 1507: 17,
 1656: 18,
 1662: 19,
 1701: 20,
 1713: 21,
 1719: 22,
 1878: 23,
 1901: 24,
 2297: 25,
 2340: 26,
 2535: 27,
 2662: 28,
 2750: 29,
 2857: 30,
 2871: 31,
 2986: 32,
 3081: 33,
 3218: 34,
 3252: 35,
 3452: 36,
 3468: 37,
 3495: 38,
 3746: 39,
 3789: 40,
 3933: 41,
 4051: 42,
 4267: 43,
 4347: 44,
 4390: 45,
 4402: 46,
 4442: 47,
 4528: 48,
 4541: 49,
 4695: 50,
 4880: 51,
 4915: 52,
 5176: 53,
 5201: 54,
 5303: 55,
 5511: 56,
 5536: 57,
 5864: 58,
 5974: 59,
 6140: 60,
 6174: 61,
 6236: 62,
 6375: 63,
 6449: 64,
 6793: 65,
 7120: 66,
 7190: 67,
 7390: 68,
 7523: 69,
 7786: 70,
 7802: 71,
 7813: 72,
 7837: 73,
 7921: 74,
 7995: 75,
 8259: 76,
 8375: 77,
 8400: 78,
 8427: 79,
 8447: 80,
 8664: 81,
 8683: 82,
 8728: 83,
 25767: 84,
 25822: 85,
 26148: 86,
 26707: 87,
 26803: 88,
 27192: 89,
 27334: 90,
 27392: 91,
 2755

In [26]:
print(index_to_user_id)

{0: 149, 1: 487, 2: 499, 3: 677, 4: 897, 5: 942, 6: 1398, 7: 1980, 8: 2023, 9: 2270, 10: 2327, 11: 2374, 12: 2401, 13: 2402, 14: 2420, 15: 2525, 16: 2844, 17: 3071, 18: 3115, 19: 3469, 20: 3589, 21: 3749, 22: 3791, 23: 3898, 24: 4092, 25: 4182, 26: 4214, 27: 4249, 28: 4605, 29: 4610, 30: 4695, 31: 4730, 32: 4856, 33: 4880, 34: 4961, 35: 4965, 36: 5283, 37: 5383, 38: 5734, 39: 5889, 40: 5987, 41: 6116, 42: 6386, 43: 6420, 44: 6450, 45: 6888, 46: 7155, 47: 7387, 48: 7716, 49: 7760, 50: 8128, 51: 8153, 52: 8227, 53: 8238, 54: 8240, 55: 8511, 56: 8583, 57: 8666, 58: 8729, 59: 8747, 60: 8828, 61: 8942, 62: 8978, 63: 9012, 64: 9125, 65: 9138, 66: 9143, 67: 9240, 68: 9246, 69: 9448, 70: 9667, 71: 9672, 72: 9726, 73: 10109, 74: 10129, 75: 10410, 76: 10622, 77: 10673, 78: 10730, 79: 11004, 80: 11133, 81: 11155, 82: 11212, 83: 11218, 84: 11329, 85: 11413, 86: 11436, 87: 11819, 88: 11870, 89: 11969, 90: 12038, 91: 12039, 92: 12155, 93: 12184, 94: 12634, 95: 12679, 96: 12685, 97: 12725, 98: 12734,

In [27]:
print(user_id_to_index)

{149: 0, 487: 1, 499: 2, 677: 3, 897: 4, 942: 5, 1398: 6, 1980: 7, 2023: 8, 2270: 9, 2327: 10, 2374: 11, 2401: 12, 2402: 13, 2420: 14, 2525: 15, 2844: 16, 3071: 17, 3115: 18, 3469: 19, 3589: 20, 3749: 21, 3791: 22, 3898: 23, 4092: 24, 4182: 25, 4214: 26, 4249: 27, 4605: 28, 4610: 29, 4695: 30, 4730: 31, 4856: 32, 4880: 33, 4961: 34, 4965: 35, 5283: 36, 5383: 37, 5734: 38, 5889: 39, 5987: 40, 6116: 41, 6386: 42, 6420: 43, 6450: 44, 6888: 45, 7155: 46, 7387: 47, 7716: 48, 7760: 49, 8128: 50, 8153: 51, 8227: 52, 8238: 53, 8240: 54, 8511: 55, 8583: 56, 8666: 57, 8729: 58, 8747: 59, 8828: 60, 8942: 61, 8978: 62, 9012: 63, 9125: 64, 9138: 65, 9143: 66, 9240: 67, 9246: 68, 9448: 69, 9667: 70, 9672: 71, 9726: 72, 10109: 73, 10129: 74, 10410: 75, 10622: 76, 10673: 77, 10730: 78, 11004: 79, 11133: 80, 11155: 81, 11212: 82, 11218: 83, 11329: 84, 11413: 85, 11436: 86, 11819: 87, 11870: 88, 11969: 89, 12038: 90, 12039: 91, 12155: 92, 12184: 93, 12634: 94, 12679: 95, 12685: 96, 12725: 97, 12734: 98,

In [28]:
item_user_array.shape

(2369, 435)

In [29]:
item_user_matrix.shape

(2369, 435)

In [30]:
# We print the item_similarity_matrix
item_similarity_matrix_train

array([[1.        , 0.        , 0.04116237, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04116237, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [31]:
item_similarity_matrix_train.shape

(435, 435)

### Step 2: Neighborhood Selection
- Determine the neighborhood size, i.e., the number of most similar items to consider when predicting ratings for a target item.
- Select the most similar items for each item in your dataset based on their calculated similarities. This forms the neighborhood for each item.

#### Item-Based Neighborhoods and Ratings Aggregation

This code following snippet enhances the previous item-based collaborative filtering approach by considering ratings aggregation within the item neighborhoods.

##### Steps:

1. **Defining Neighborhood Size**:
   - The variable `neighborhood_size` determines the number of most similar items to consider in the neighborhood.

2. **Initializing Data Structure**:
   - An empty dictionary `item_neighborhoods` is initialized to store the neighborhoods for each item.

3. **Iterating Over Items**:
   - For each movie in the dataset:
     - All ratings for the current movie are extracted from the DataFrame (`df`).
     - Ratings aggregation is performed. In this example, the average rating for the movie is computed, but other aggregation methods can be used.
     - The similarity scores for the current movie are retrieved from the precomputed `item_similarity_matrix`.
     - Similarity scores are sorted in descending order, and the indices of the most similar items (excluding itself) are obtained.
     - These indices are converted back to movie IDs, forming the neighborhood for the current item.
     - The neighborhood for the current item is stored in the `item_neighborhoods` dictionary.

4. **Output**:
   - `item_neighborhoods`: A dictionary where keys are movie IDs, and values are lists of movie IDs representing the neighborhood of each item. Each movie's neighborhood includes movies with similar ratings and content.

##### Note:
- This approach considers both similarity in ratings and content (as captured by cosine similarity) when building item neighborhoods.
- Aggregating ratings within item neighborhoods helps in providing more personalized recommendations.
- The choice of rating aggregation method (e.g., mean, median) can impact the quality of recommendations and may need to be adjusted based on the characteristics of the dataset and user preferences.


In [32]:
import numpy as np

def neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix, neighborhood_size=5):
    # Initialize an empty dictionary to store item neighborhoods
    item_neighborhoods = {}

    # Iterate over each item (movie) index in the dataset
    for movie_index in range(item_user_array.shape[1]):
        # Convert the item index to movie ID
        movie_id = index_to_movie_id[movie_index]

        # Extract all ratings for the current movie
        movie_ratings = item_user_array[:, movie_index]

        # Aggregate ratings (e.g., compute the mean rating)
        movie_avg_rating = np.mean(movie_ratings)

        # Retrieve similarity scores for the current movie
        similarity_scores = item_similarity_matrix[movie_index]

        # Sort similarity scores in descending order and get indices of most similar items
        most_similar_indices = np.argsort(similarity_scores)[::-1][1:neighborhood_size+1]

        # Convert indices back to movie IDs to form the neighborhood
        neighborhood = [index_to_movie_id[idx] for idx in most_similar_indices]

        # Store the neighborhood for the current item in the item_neighborhoods dictionary
        item_neighborhoods[movie_id] = neighborhood

    return item_neighborhoods

# Example usage:
item_neighborhoods_train = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train)
print(item_neighborhoods_train)


{75: [153608, 165059, 163014, 32128, 161778], 77: [128572, 180989, 136786, 136455, 135629], 135: [139590, 136786, 136455, 135629, 189011], 208: [201686, 113176, 60032, 86420, 83435], 290: [177339, 165059, 78463, 170311, 171173], 413: [45072, 195247, 195645, 31353, 157637], 461: [161426, 31353, 167700, 165059, 163014], 733: [128572, 149310, 27671, 27771, 195247], 792: [161426, 31353, 167700, 165059, 163014], 810: [153608, 31353, 195247, 195645, 161778], 857: [68319, 61323, 61473, 62254, 62437], 901: [32128, 165059, 27823, 163014, 31353], 955: [205913, 139590, 45072, 5176, 139277], 1184: [132144, 32128, 195247, 195645, 157637], 1221: [153608, 32128, 109385, 31353, 27823], 1352: [135629, 143285, 191381, 139590, 139277], 1365: [184043, 184287, 163014, 124769, 122406], 1507: [149310, 161778, 195247, 195645, 161426], 1656: [139277, 136786, 62254, 184043, 136455], 1662: [191381, 135629, 180989, 62254, 134122], 1701: [191381, 45072, 193695, 5176, 143285], 1713: [177339, 27671, 165059, 116521, 

### Step 4: Rating Prediction

- For each target item and user pair where the user hasn't rated the target item:
Identify the neighborhood of similar items to the target item.
- Predict the rating for the target item using a weighted average of the ratings of the items in its neighborhood, where the weights are the similarities between the items and the target item.
- Adjust the prediction based on the user's average rating or other normalization techniques, if necessary.

In [33]:
# Train your model and create the item_neighborhoods dictionary using the training data (steps 1-3)

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_train = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in train_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_train.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_train.fillna(predicted_ratings_train.mean().mean(), inplace=True)

# Display the head of the predicted_ratings DataFrame
predicted_ratings_train.head()


C:\Users\Jaume\AppData\Local\Temp\ipykernel_9384\1268806476.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_train.fillna(predicted_ratings_train.mean().mean(), inplace=True)


movieId,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
userId,,,,,,,,,,,,,,,,,,,,,
214831,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,...,2.062500,2.000000,2.000000,2.000000,2.000000,2.062500,2.062500,2.062500,2.062500,2.062500
8942,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,...,2.800000,2.000000,2.000000,2.000000,2.000000,2.800000,2.800000,2.800000,2.800000,2.800000
189543,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,...,3.166667,3.000000,3.000000,3.000000,3.000000,3.166667,3.166667,3.166667,3.166667,3.166667
34677,3.416667,3.416667,3.416667,3.416667,3.416667,3.416667,3.416667,3.416667,3.416667,3.416667,...,3.416667,3.000000,3.000000,3.000000,3.000000,3.416667,3.416667,3.416667,3.416667,3.416667
128377,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,3.378035,...,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333


In [34]:
# We count the NaN values in the predicted_ratings_train dataframe
predicted_ratings_train.isnull().sum().sum()

0

In [35]:
def get_top_recommendations(user_id, predicted_ratings, df):
    """
    Get top movie recommendations for a given user using predicted ratings.

    Parameters:
        user_id (int): ID of the user for whom recommendations are to be generated.
        predicted_ratings (pd.DataFrame): DataFrame containing predicted ratings for users and movies.
        df (pd.DataFrame): DataFrame containing movie ratings.

    Returns:
        list: Top movie titles recommended for the user.
    """
    # Ensure that predicted_ratings is a DataFrame
    if not isinstance(predicted_ratings, pd.DataFrame):
        raise ValueError("predicted_ratings must be a pandas DataFrame")

    # Check if the user ID exists in the predicted ratings DataFrame's index
    if user_id not in predicted_ratings.index:
        # If the user ID doesn't exist, return an empty list
        return ["The user doesn't exist"]

    # Get the predicted ratings for the user
    user_predicted_ratings = predicted_ratings.loc[user_id]
    
    # Filter out the movies that the user has already seen
    seen_movies = set(df[df['userId'] == user_id]['movieId'])
    unseen_movies = [movie_id for movie_id in user_predicted_ratings.index if movie_id not in seen_movies]
    
    # Check if there are unseen movies with predicted ratings
    if not unseen_movies:
        # If all movies have been seen, return an empty list
        return []
    
    # Sort the unseen movies by predicted rating in descending order
    sorted_unseen_movies = user_predicted_ratings[unseen_movies].sort_values(ascending=False)
    
    # Get the top 5 movie titles
    top_movie_ids = sorted_unseen_movies.head(5).index.tolist()
    
    # Get the unique movie titles corresponding to the top movie IDs
    top_movie_titles = set(df[df['movieId'].isin(top_movie_ids)]['title'])
    
    return list(top_movie_titles)[:5]  # Return only the first 5 unique movie titles


In [36]:
top_recommendations = []

# Iterate over each row (user) in the train_matrix
for user_id in train_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_train, df)
    top_recommendations.append(recommendations)

# Output: List of top movie recommendations for each user in the train matrix
top_recommendations

# print size of top_recommendations
len(top_recommendations)


[['Twisted Seduction ',
  'Job, czyli ostatnia szara komórka ',
  'Big Bully ',
  'Going By The Book ',
  'Frankenstein '],
 ['Twisted Seduction ',
  'Job, czyli ostatnia szara komórka ',
  'Big Bully ',
  'Frankenstein ',
  'Pandemic '],
 ['Twisted Seduction ',
  'Job, czyli ostatnia szara komórka ',
  'Big Bully ',
  'Frankenstein ',
  'Pandemic '],
 ['Requiem for the American Dream ',
  'The Grinch Grinches the Cat in the Hat ',
  'Big Bully ',
  'Frankenstein ',
  'Pandemic '],
 ['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Te3n ',
  'Horton Hears a Who! ',
  'Lorenzo '],
 ['Uncle P ', 'Mascots ', 'Big Bully ', 'Horton Hears a Who! ', 'Lorenzo '],
 ['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Silver Lode ',
  'Horton Hears a Who! ',
  'Lorenzo '],
 ['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Silver Lode ',
  'Horton Hears a Who! ',
  'Lorenzo '],
 ['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Te3n ',
  'Horton Hears a Who! 

1516

In [37]:
print("Last version")

Last version


### Step 5: Model Evaluation
- We evaluate the performance of your ItemKNN algorithm using appropriate evaluation metrics such as Root Mean Squared Error (RMSE), Mean Absolute Error (MAE), or others.
- Split your dataset into training and testing sets to assess the model's predictive accuracy on unseen data.

In [38]:
train.shape

predicted_ratings_train.shape

(1516, 435)

(1516, 435)

In [39]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(train_matrix, predicted_ratings):
    """
    Evaluate the model's performance on the training data.

    Parameters:
        train_matrix (numpy.ndarray): Item-user matrix from the training data.
        predicted_ratings (pandas.DataFrame or numpy.ndarray): Predicted ratings DataFrame or array for the training data.

    Returns:
        float: Mean Absolute Error (MAE) on the training data.
        float: Root Mean Squared Error (RMSE) on the training data.
    """
    # Convert predicted_ratings to a numpy array if it's a DataFrame
    if isinstance(predicted_ratings, pd.DataFrame):
        predicted_ratings = predicted_ratings.to_numpy()

    # Ensure train_matrix and predicted_ratings have the same shape
    assert train_matrix.shape == predicted_ratings.shape, "Shapes of train_matrix and predicted_ratings are not consistent."

    # Initialize lists to store true and predicted ratings
    true_ratings = []
    pred_ratings = []

    # Iterate over each user and their ratings
    for user_id, user_ratings in enumerate(train_matrix):
        for movie_id, rating in enumerate(user_ratings):
            # Skip unrated movies
            if rating == 0:
                continue

            # Check if the indices are within the bounds of the predicted ratings array
            if user_id < predicted_ratings.shape[0] and movie_id < predicted_ratings.shape[1]:
                # Get the predicted rating for the corresponding movie
                pred_rating = predicted_ratings[user_id, movie_id]

                # Append the true and predicted ratings
                true_ratings.append(rating)
                pred_ratings.append(pred_rating)

    # Convert lists to numpy arrays
    true_ratings = np.array(true_ratings)
    pred_ratings = np.array(pred_ratings)

    # Calculate evaluation metrics
    mae = mean_absolute_error(true_ratings, pred_ratings)
    rmse = np.sqrt(mean_squared_error(true_ratings, pred_ratings))

    return mae, rmse

# Assuming train is the numpy array representing the item-user matrix
# Assuming predicted_ratings_train is the DataFrame or numpy array representing the predicted ratings for the training data

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.8408671530283619
Root Mean Squared Error (RMSE) on Training Data: 1.0585541515836878


### Step 6: Parameter Tuning
- Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of your ItemKNN algorithm.
- Use cross-validation or other techniques to tune these parameters and avoid overfitting.

1. **Import Necessary Libraries:**
   - We import the required libraries for performing grid search cross-validation (`GridSearchCV`), creating custom scorers (`make_scorer`), and utilizing the `NearestNeighbors` algorithm.

2. **Define Cosine Similarity Function:**
   - We define a custom function `cosine_similarity` to compute the cosine similarity between two vectors. This function calculates the dot product of the vectors and divides it by the product of their norms.

3. **Define Custom Scorer:**
   - We create a custom scorer `cosine_similarity_scorer` using `make_scorer`, which enables us to use cosine similarity as the scoring metric during grid search cross-validation.

4. **Define Parameter Grid:**
   - We specify a parameter grid `param_grid` containing the hyperparameters to be tuned. In this case, we're tuning the number of neighbors (`n_neighbors`) and the distance metric (`metric`) for the `NearestNeighbors` algorithm.

5. **Initialize NearestNeighbors Model:**
   - We initialize the `NearestNeighbors` model without specifying any hyperparameters.

6. **Create GridSearchCV Object:**
   - We create a `GridSearchCV` object named `grid_search` with the specified parameter grid, cross-validation strategy (5-fold cross-validation), and custom scoring metric (`cosine_similarity_scorer`).

7. **Fit the Data:**
   - We fit the `item_user_matrix` data to the `grid_search` object to perform hyperparameter tuning. `item_user_matrix` typically contains the item-item similarity matrix computed using collaborative filtering techniques.

8. **Get Best Hyperparameters:**
   - After fitting the data, we retrieve the best hyperparameters selected by the grid search using the `best_params_` attribute of the `grid_search` object.

9. **Print Best Parameters:**
   - Finally, we print the best hyperparameters obtained from the grid search.



`scikit-learn (sklearn)`:

- Scikit-learn is a popular machine learning library in Python that provides simple and efficient tools for data analysis and modeling.
- It includes various modules for tasks such as classification, regression, clustering, dimensionality reduction, and model selection.
- The GridSearchCV class from scikit-learn is used for hyperparameter tuning through grid search along with cross-validation.
- The make_scorer function allows you to create a custom scoring function for use with GridSearchCV.
- The NearestNeighbors class provides functionality for unsupervised nearest neighbors learning, which can be used for tasks such as finding k-nearest neighbors for a given data point.

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # train_matrix contains item-user matrix for the train set

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


This code sets up a GridSearchCV object to perform hyperparameter tuning using the train set (train_matrix). It explores different combinations of hyperparameters specified in the param_grid, evaluates them using 5-fold cross-validation (cv=5), and uses the cosine_similarity scorer to optimize the model's performance based on cosine similarity. Finally, it prints the best hyperparameters found during the search.

We have to perform the calculation of the predicted ratings for the train-val set to generate predicted ratings in order to evaluate the performance of our trained model on a validation dataset. 

If we computed the similarity matrix again specifically for the train_val set, it would essentially mean that we are using a different set of similarity measures for predicting ratings compared to what we used during training. This approach could lead to inconsistencies and potentially degrade the performance of our model. Here's what could happen:

1. **Inconsistency**: The similarity measures computed for the train_val set might differ from those computed for the training set due to variations in the data. As a result, the predicted ratings based on these new similarity measures may not align well with the predictions made during training, leading to inconsistency in the model's behavior.

2. **Overfitting**: Computing a new similarity matrix specifically for the train_val set might lead to overfitting on the validation data. The model may capture noise or idiosyncrasies present in the train_val set, which may not generalize well to unseen data.

3. **Increased Complexity**: Computing the similarity matrix again for the train_val set adds computational complexity and redundancy, especially if the similarity computation process is resource-intensive. This can result in longer training times and increased resource utilization.

Overall, it's generally recommended to use the same similarity measures or neighborhood definitions for both training and validation sets to ensure consistency and generalizability of the model.

In [41]:
# Assuming you have already trained your model and obtained the item_neighborhoods_train dictionary

# Initialize an empty DataFrame to store predicted ratings for train_val
predicted_ratings_val = pd.DataFrame(index=val_matrix.index, columns=val_matrix.columns)

# Iterate over each user and their ratings in the train_val set
for user_id, user_ratings in val_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the train set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_val.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_val.fillna(predicted_ratings_val.mean().mean(), inplace=True)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_9384\2864728636.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_val.fillna(predicted_ratings_val.mean().mean(), inplace=True)


In [42]:
predicted_ratings_val.head()

predicted_ratings_val.shape

movieId,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
userId,,,,,,,,,,,,,,,,,,,,,
129634,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.361793,3.500000,3.500000,...,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000
245003,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,...,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333
33227,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
169378,4.375000,4.375000,4.375000,4.375000,3.361793,4.375000,4.375000,3.361793,4.375000,4.375000,...,4.375000,4.375000,4.375000,4.375000,4.375000,4.375000,4.375000,4.375000,4.375000,4.375000
21387,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,...,4.333333,3.500000,3.500000,3.500000,3.500000,4.333333,4.333333,4.333333,4.333333,4.333333


(379, 435)

Now we evaluate the model with the Validation item-user matrix.

In [43]:
# Now, you can evaluate the model using the evaluate_model function
mae, rmse = evaluate_model(val_matrix.to_numpy(), predicted_ratings_val)

print("Mean Absolute Error (MAE) on Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Validation Data:", rmse)


Mean Absolute Error (MAE) on Validation Data: 0.8698448811630014
Root Mean Squared Error (RMSE) on Validation Data: 1.097606881805091


### Retraining of the model with the Train-Val set

In [44]:
# After the model evaluation on the validation set, we recalculate the similarity matrix
# of the model on the train-val data
item_similarity_matrix_train_val = calculate_item_similarity_matrix(train_val)

# We calculate the item neighborhoods for the train_val data 
item_neighborhoods_train_val = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train_val)

C:\Users\Jaume\AppData\Local\Temp\ipykernel_9384\2934347206.py:6: RuntimeWarning: invalid value encountered in scalar divide
  similarity = dot_product / (norm_a * norm_b)


### Step 7: Deployment

Once we have come up with the best parameters possible and trained the model with the whole train_validation set, we will test it on the test set.

In [45]:
test_matrix

movieId,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
userId,,,,,,,,,,,,,,,,,,,,,
150403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
247780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
# Check if there are NaN values in the test matrix
test_matrix.isnull().sum().sum()

0

In [47]:
# Now, we can evaluate the model's predicted ratings on the test set

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_test = pd.DataFrame(index=test_matrix.index, columns=test_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in test_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train_val:
            neighborhood = item_neighborhoods_train_val[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_test.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)

# Now, you can evaluate the model's predicted ratings on the test set
mae, rmse = evaluate_model(test_matrix.to_numpy(), predicted_ratings_test.to_numpy())

print("Mean Absolute Error (MAE) on Test Data using the Ratings given to the Test Set:", mae)
print("Root Mean Squared Error (RMSE) on Test Data using the Ratings given to the Test Set:", rmse)


Mean Absolute Error (MAE) on Test Data using the Ratings given to the Test Set: 0.8665656419289083
Root Mean Squared Error (RMSE) on Test Data using the Ratings given to the Test Set: 1.0846514049016567


C:\Users\Jaume\AppData\Local\Temp\ipykernel_9384\2872005839.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)


In [48]:
top_recommendations_test = []

# Iterate over each row (user) in the test_matrix
for user_id in test_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_test, df)
    top_recommendations_test.append(recommendations)

# Output: List of top movie recommendations for each user in the test matrix
top_recommendations_test

# print size of top_recommendations
print("Size of top_recommendations_test:", len(top_recommendations_test))



[['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Te3n ',
  'Horton Hears a Who! ',
  'Lorenzo '],
 ['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Te3n ',
  'Horton Hears a Who! ',
  'Lorenzo '],
 ['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Silver Lode ',
  'Horton Hears a Who! ',
  'Lorenzo '],
 ['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Te3n ',
  'Horton Hears a Who! ',
  'Lorenzo '],
 ['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Silver Lode ',
  'Horton Hears a Who! ',
  'Lorenzo '],
 ['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Te3n ',
  'Horton Hears a Who! ',
  'Lorenzo '],
 ['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Te3n ',
  'Horton Hears a Who! ',
  'Lorenzo '],
 ['Pursuit of Happyness, The ',
  'Beastmaster, The ',
  'Once Were Warriors ',
  'Waterworld ',
  'Airheads '],
 ['Charles Darwin and the Tree of Life ',
  'Big Bully ',
  'Te3n ',
  'Horton Hears a Who! ',
  'Loren

Size of top_recommendations_test: 474


# Item-based Classification (ItemKNN)

### Step 1: Data preparation
- Ensure you have a dataset with user-item interactions. Each interaction should include the user ID, item ID, and the corresponding label or class for classification.
- Preprocess the data as needed, including handling missing values, encoding categorical variables, and splitting into training and testing sets.

For this step we are going to resuse the `item_user_matrix` that we created in the first model built. 

### Step 2: Compute Item Similarity


In [49]:
item_similarity_matrix_train.shape

(435, 435)

In [50]:
import numpy as np

def item_neighborhood_selection(similarity_matrix, k=None, threshold=None, item_ids=None):
    """
    Select a subset of similar items for each item based on similarity matrix.

    Parameters:
        similarity_matrix (numpy.ndarray): Item-item similarity matrix.
        k (int): Number of similar items to select (optional).
        threshold (float): Similarity threshold for selecting similar items (optional).
        item_ids (list): List of item IDs corresponding to rows/columns of the similarity matrix.

    Returns:
        dict: Dictionary containing similar items for each item.
    """
    num_items = similarity_matrix.shape[0]
    item_neighborhood = {}

    for i in range(num_items):
        if k is not None:
            # Select top-k similar items (excluding the item itself)
            similar_items_indices = np.argsort(similarity_matrix[i])[::-1][:k]
        elif threshold is not None:
            # Select items with similarity above threshold (excluding the item itself)
            similar_items_indices = np.where(similarity_matrix[i] > threshold)[0]

        # Remove the item itself from the neighborhood
        similar_items_indices = similar_items_indices[similar_items_indices != i]

        # Get the item ID corresponding to the current index
        current_item_id = item_ids[i] if item_ids is not None else i

        # Get the item IDs for the neighborhood
        neighborhood_item_ids = [item_ids[index] for index in similar_items_indices]

        # Store the similar items with the item's ID as the key
        item_neighborhood[current_item_id] = neighborhood_item_ids

    return item_neighborhood

# Example usage:
# Assuming you have an item-item similarity matrix 'item_similarity_matrix_train'
# and you want to select top-5 similar items for each item
k = 5
item_neighborhoods_classification_train = item_neighborhood_selection(item_similarity_matrix_train, k=k, item_ids=index_to_movie_id)

# Check if there are any items with empty neighborhoods
empty_neighborhoods = []

for item_id, neighborhood in item_neighborhoods_classification_train.items():
    if not neighborhood:
        empty_neighborhoods.append(item_id)

if empty_neighborhoods:
    print("Items with empty neighborhoods:", empty_neighborhoods)
else:
    print("No items with empty neighborhoods found.")


No items with empty neighborhoods found.


In [51]:
len(item_neighborhoods_classification_train)

435

### Step 3: Ratings classification selection

In [52]:
import numpy as np
import pandas as pd

def recommend_movies(train_matrix, item_neighborhoods, movie_id_to_index, num_movies=5):
    """
    Recommend top movies for each user based on item neighborhood.

    Parameters:
        train_matrix (np.ndarray or pd.DataFrame): Matrix containing user-item ratings in the train set.
        item_neighborhoods (dict): Dictionary containing similar items for each item.
        movie_id_to_index (dict): Dictionary mapping movie IDs to their corresponding indices in the train_matrix.
        num_movies (int): Number of movies to recommend for each user.

    Returns:
        pd.DataFrame: DataFrame containing predicted ratings for recommended movies for each user.
    """
    # Convert train_matrix to DataFrame if it's a numpy array
    if isinstance(train_matrix, np.ndarray):
        train_matrix = pd.DataFrame(train_matrix, index=np.arange(train_matrix.shape[0]), columns=np.arange(train_matrix.shape[1]))

    # Initialize an empty DataFrame to store predicted ratings
    predicted_ratings = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

    # Iterate over each user-item pair in the training set
    for user_id, user_ratings in train_matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            
            # Skip if the rating is non-zero (indicating a rating given by the user)
            if rating != 0:
                continue
            
            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]
                
                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False
                
                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break
                
                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = train_matrix[train_matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()
                
                # Assign the predicted rating to the corresponding cell in the DataFrame
                predicted_ratings.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)

    # Set the index of the DataFrame as the user ID of the train set
    predicted_ratings.index = train_matrix.index

    # Display the head of the predicted_ratings DataFrame
    print(predicted_ratings.head())

    return predicted_ratings

# Example usage:
# Assuming you have the train matrix, item neighborhood dictionary, and movie_id_to_index dictionary
# train_matrix, item_neighborhoods, movie_id_to_index = ...

# Recommend movies for each user in the train set
predicted_ratings_classification_train = recommend_movies(train_matrix, item_neighborhoods_classification_train, movie_id_to_index)

# Print the shape of the predicted ratings matrix
print("Shape of predicted ratings matrix:", predicted_ratings_classification_train.shape)


movieId    75        77        135       208       290       413       461     \
userId                                                                          
214831   2.062500  2.062500  2.062500  2.062500  2.062500  2.062500  2.062500   
8942     2.800000  2.800000  2.800000  2.800000  2.800000  2.800000  2.800000   
189543   3.166667  3.166667  3.166667  3.166667  3.166667  3.166667  3.166667   
34677    3.416667  3.416667  3.416667  3.416667  3.416667  3.416667  3.416667   
128377   2.833333  2.833333  2.833333  2.833333  2.833333  2.833333  2.833333   

movieId    733       792       810     ...    271162    272579    274245  \
userId                                 ...                                 
214831   2.062500  2.062500  2.062500  ...  2.062500  2.062500  2.062500   
8942     2.800000  2.800000  2.800000  ...  2.800000  2.000000  2.000000   
189543   3.166667  3.166667  3.166667  ...  3.166667  3.000000  3.000000   
34677    3.416667  3.416667  3.416667  ...  3.416667

C:\Users\Jaume\AppData\Local\Temp\ipykernel_9384\1893350747.py:61: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)


In [53]:
type(train_matrix)

pandas.core.frame.DataFrame

In [54]:
predicted_ratings_classification_train

movieId,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
userId,,,,,,,,,,,,,,,,,,,,,
214831,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,...,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500,2.062500
8942,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,...,2.800000,2.000000,2.000000,2.000000,2.000000,2.800000,2.800000,2.800000,2.800000,2.800000
189543,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,...,3.166667,3.000000,3.000000,3.000000,3.000000,3.166667,3.166667,3.166667,3.166667,3.166667
34677,3.416667,3.416667,3.416667,3.416667,3.416667,3.416667,3.416667,3.416667,3.416667,3.416667,...,3.416667,3.000000,3.000000,3.000000,3.000000,3.416667,3.416667,3.416667,3.416667,3.416667
128377,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,3.377853,...,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333,2.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232128,3.166667,3.166667,3.166667,3.377853,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,...,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667
166909,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.377853,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
325458,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000


### Step 4: Recommendations Generation

In [55]:
import pandas as pd

# Convert predicted_ratings_classification_train to a pandas DataFrame if it's a numpy array
if isinstance(predicted_ratings_classification_train, np.ndarray):
    # Use the length of train_matrix as the number of rows and items as columns
    num_users, num_items = train_matrix.shape
    predicted_ratings_classification_train = pd.DataFrame(predicted_ratings_classification_train, index=range(num_users), columns=range(num_items))

# Initialize a dictionary to store top recommendations for each user
top_recommendations_per_user = {}

# Iterate over each user in the train matrix
for user_id in train_matrix.index:
    # Get the top recommendations for the current user using predicted_ratings_classification_train
    recommendations = get_top_recommendations(user_id, predicted_ratings_classification_train, df)
    
    # Store the recommendations in the dictionary
    top_recommendations_per_user[user_id] = recommendations

# Print the top 5 recommendations for each user
for user_id, recommendations in top_recommendations_per_user.items():
    print(f"User {user_id} recommendations:", recommendations)


User 214831 recommendations: ['Charles Darwin and the Tree of Life ', 'Uncle P ', 'Big Bully ', 'Matchmaking ', 'Lorenzo ']
User 8942 recommendations: ['Twisted Seduction ', 'Job, czyli ostatnia szara komórka ', 'Big Bully ', 'Frankenstein ', 'Pandemic ']
User 189543 recommendations: ['Twisted Seduction ', 'Job, czyli ostatnia szara komórka ', 'Big Bully ', 'Frankenstein ', 'Pandemic ']
User 34677 recommendations: ['Requiem for the American Dream ', 'The Grinch Grinches the Cat in the Hat ', 'Big Bully ', 'Frankenstein ', 'Pandemic ']
User 128377 recommendations: ['Charles Darwin and the Tree of Life ', 'Big Bully ', 'Te3n ', 'Horton Hears a Who! ', 'Lorenzo ']
User 46734 recommendations: ['Uncle P ', 'Mascots ', 'Big Bully ', 'Horton Hears a Who! ', 'Lorenzo ']
User 270802 recommendations: ['Charles Darwin and the Tree of Life ', 'Big Bully ', 'Silver Lode ', 'Horton Hears a Who! ', 'Lorenzo ']
User 72387 recommendations: ['Charles Darwin and the Tree of Life ', 'Big Bully ', 'Silver 

### Step 5: Model Evaluation (Item KNN Classification)

In [56]:
import numpy as np
import pandas as pd

# Assuming predicted_ratings_classification_train is your predicted ratings DataFrame or numpy array

# Ensure predicted_ratings_classification_train is a numpy array
if isinstance(predicted_ratings_classification_train, pd.DataFrame):
    predicted_ratings_classification_train = predicted_ratings_classification_train.to_numpy()

# Assuming train_matrix is the numpy array representing the item-user matrix

# Ensure train_matrix is a 2D array
if isinstance(train_matrix, pd.DataFrame):
    train_matrix = train_matrix.to_numpy()

if train_matrix.ndim == 1:
    train_matrix = np.expand_dims(train_matrix, axis=0)

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_classification_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.8408892018985091
Root Mean Squared Error (RMSE) on Training Data: 1.0585474377233977


### Step 6: Parameter Tuning

Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of your ItemKNN algorithm.
Use techniques like cross-validation to tune these parameters and avoid overfitting.

In [57]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # val_matrix contains item-user matrix

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


We are now going to obtain again the ratings for the full val set to evaluate the performance. 

In [58]:
# Assuming you have the best model after hyperparameter tuning
# best_model = ...

# Function to recalculate predictions for a given matrix using the trained model
def recalculate_predictions(matrix, item_neighborhoods):
    """
    Recalculate predictions for a given matrix using the trained model.

    Parameters:
        matrix (pd.DataFrame): Matrix containing user-item ratings.

    Returns:
        pd.DataFrame: DataFrame containing recalculated predicted ratings for each user.
    """
    # Initialize an empty DataFrame to store recalculated predicted ratings
    recalculated_predictions = pd.DataFrame(index=matrix.index, columns=matrix.columns)

    # Iterate over each user-item pair in the matrix
    for user_id, user_ratings in matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            # Skip if the rating is non-zero (indicating a rating given by the user)
            if rating != 0:
                continue

            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]

                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False

                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break

                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = matrix[matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()

                # Assign the predicted rating to the corresponding cell in the DataFrame
                recalculated_predictions.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)

    return recalculated_predictions


# Recalculate predictions for the validation set (train_val_matrix)
recalculated_val_predictions = recalculate_predictions(val_matrix, item_neighborhoods_classification_train)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_9384\4246365781.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


In [59]:
recalculated_val_predictions

movieId,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
userId,,,,,,,,,,,,,,,,,,,,,
129634,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.360448,3.500000,3.500000,...,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000
245003,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,...,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333
33227,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
169378,4.375000,4.000000,4.375000,4.375000,3.360448,4.375000,4.000000,3.360448,4.000000,4.375000,...,4.375000,4.375000,4.375000,4.375000,4.375000,4.000000,4.375000,4.375000,4.000000,4.375000
21387,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,...,4.333333,3.500000,3.500000,3.500000,3.500000,4.333333,4.333333,4.333333,4.333333,4.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269908,4.000000,4.000000,4.000000,4.000000,3.360448,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
298574,2.300000,2.300000,2.300000,2.300000,2.300000,2.300000,2.300000,2.300000,2.300000,2.300000,...,2.300000,2.300000,2.300000,2.300000,2.300000,2.300000,2.300000,2.300000,2.300000,2.300000
95283,2.625000,2.625000,2.625000,3.360448,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,...,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000,2.625000


In [60]:
# Evaluate the model
mae, rmse = evaluate_model(val, recalculated_val_predictions)

print("Mean Absolute Error (MAE) on Train-Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Train-Validation Data:", rmse)

Mean Absolute Error (MAE) on Train-Validation Data: 0.8700529749080804
Root Mean Squared Error (RMSE) on Train-Validation Data: 1.0975843103210317


### Step 7: Deployment

Once we have all the metrics, we have to recalculate the ratings for the test set, we are going to repeat the same process for the test set.

In [61]:
# We have to reobtain the neighborhood for the train_val data
# We will use the already calculated item-item similarity matrix for train_val data
item_neighborhoods_classification_train_val = item_neighborhood_selection(item_similarity_matrix_train_val, k=5, item_ids=index_to_movie_id)

# Recalculate predictions for the validation set (test matrix)
recalculated_test_predictions = recalculate_predictions(test_matrix, item_neighborhoods_classification_train_val)

C:\Users\Jaume\AppData\Local\Temp\ipykernel_9384\4246365781.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


We also have to evaluate the metrics for the test set.

In [62]:
# Evaluate the model
mae, rmse = evaluate_model(test, recalculated_test_predictions)

print("Mean Absolute Error (MAE) on Test Data:", mae)
print("Root Mean Squared Error (RMSE) on Test Data:", rmse)

Mean Absolute Error (MAE) on Test Data: 0.8665836795534453
Root Mean Squared Error (RMSE) on Test Data: 1.0846347368024298
